# Lab3


# Задание 1


Пусть есть две независимые выборки $X = [X_i \sim \cal{N} (\mu_1, \sigma_1^2)]_{i=1}^n$ и $Y = [Y_j  \sim \cal{N} (\mu_2, \sigma_2^2)]_{j=1}^m$.

Построим доверительный интервал уровня $1 - \alpha$ для параметра $\tau = \frac{\sigma_1^2}{\sigma_2^2}$ в условиях неизвестных $\mu_1, \mu_2$.

Так как смещения неизвестны, нам придётся использовать выборочную, смещённую дисперсию.

Рассмотрим статистику $T$:

$$ T = \frac{n(m-1)Var(X)}{m(n-1) Var(Y)}$$

Дисперсии, которые содержатся в статистике -- смещённые, обозначим их $S^2_X = Var(X)$ и $S^2_Y = Var(Y)$.

Введём несмещённые варианты $s^2_X = \frac{n}{n-1} S^2_X$ и $s^2_Y = \frac{m}{m-1} S^2_Y$.

Используя данную зависимость выражений, перепишем $T$ в терминах $s^2$:

$$ T = \frac{n(m-1) S^2_X}{m(n-1) S^2_Y} = \frac{n(m-1) \frac{n-1}{n} s^2_X}{m(n-1) \frac{m-1}{m} s^2_Y} = \frac{s^2_X}{s^2_Y}$$

В заданных условия справедлива теорема Фишера:

$$ \frac{(n-1) s^2}{\sigma^2} \sim \chi^2_{n-1} \leftrightarrow \frac{s^2}{\sigma^2} \sim \frac{\chi^2_{n-1}}{n-1}$$

Тут то мы и теряем одну степень свободы из-за неизвестных смещений.

Тогда теорема Фишера приводит нас к распределению Фишера:

$$ \frac{s^2_X / \sigma^2_1}{s^2_Y / \sigma^2_2} = \frac{s^2_X}{s^2_Y} \frac{\sigma^2_2}{\sigma^2_1} = T / \tau \sim \text{F}(n-1, m-1)$$

То есть получается, чтобы построить требуемый доверительный интервал нам необходимо обратиться к квантилям распределения Фишера ($F_\beta$). Данное распределение непрерывно и не зависит от $\tau$, тогда:

$$ P (F_{\alpha/2} \leq \frac{T}{\tau} \leq F_{1 -\alpha/2}) = 1 - \alpha$$
$$ P (\frac{T}{F_{1- \alpha/2}} \leq \tau \leq \frac{T}{F_{\alpha/2}})$$

Доверительный интервал уровня $1 - \alpha$:

$$ \tau \in [\frac{T}{F_{1 -\alpha/2}}; \frac{T}{F_{\alpha/2}}]$$

In [16]:
import numpy as np
from scipy.stats import f

mu1 = 0.0
mu2 = 0.0
sigma1 = np.sqrt(2.0)
sigma2 = 1.0
alpha = 0.05
true_tau = (sigma1**2) / (sigma2**2)

rng = np.random.default_rng(42)

def coverage_rate_tau(n: int, N: int, chunk_bytes: int = 64_000_000) -> float:
    max_k = max(1, int(chunk_bytes // (16 * n)))
    remain = N
    covered = 0
    F_a2 = f.isf(alpha/2, dfn=n-1, dfd=n-1)
    F_1ma2 = f.isf(1 - alpha/2, dfn=n-1, dfd=n-1)
    while remain > 0:
        k = min(remain, max_k)
        X = rng.normal(loc=mu1, scale=sigma1, size=(k, n))
        Y = rng.normal(loc=mu2, scale=sigma2, size=(k, n))
        s2_x = X.var(axis=1, ddof=1)
        s2_y = Y.var(axis=1, ddof=1)
        T = s2_x / s2_y
        left = T / F_1ma2
        right = T / F_a2
        lo = np.minimum(left, right)
        hi = np.maximum(left, right)
        covered += np.count_nonzero((lo <= true_tau) & (true_tau <= hi))
        remain -= k
    return covered / N

n_list = [25, 50, 100, 1000]
N1 = 1000
N2 = 10000

print("Покрытие 95%-ДИ для τ:")
for n in n_list:
    p1 = coverage_rate_tau(n, N1)
    p2 = coverage_rate_tau(n, N2)
    se1 = np.sqrt(max(p1*(1-p1), 1e-12) / N1)
    se2 = np.sqrt(max(p2*(1-p2), 1e-12) / N2)
    print(f"n=m={n:4d} | N={N1:5d}: {p1:.4f}  (±{1.96*se1:.4f} error);  N={N2:5d}: {p2:.4f}  (±{1.96*se2:.4f} error)")


Покрытие 95%-ДИ для τ:
n=m=  25 | N= 1000: 0.9520  (±0.0132 error);  N=10000: 0.9484  (±0.0043 error)
n=m=  50 | N= 1000: 0.9590  (±0.0123 error);  N=10000: 0.9543  (±0.0041 error)
n=m= 100 | N= 1000: 0.9510  (±0.0134 error);  N=10000: 0.9476  (±0.0044 error)
n=m=1000 | N= 1000: 0.9510  (±0.0134 error);  N=10000: 0.9501  (±0.0043 error)
n=m=1000 | N= 1000: 0.9510  (±0.0134 error);  N=10000: 0.9501  (±0.0043 error)



Монте‑Карло‑эксперимент для оценки покрываемости 95%-доверительного интервала для отношения дисперсий 
$\tau=\sigma_1^2/\sigma_2^2$ при $n=m=25$, $\mu_1=\mu_2=0$, $\sigma_1^2=2$, $\sigma_2^2=1$, истинное $\tau=2$.

- Для 1000 повторов оценка покрываемости колеблется вокруг 0.95, а доверительный интервал Монте‑Карло шире.
- Для 10000 повторов оценка покрываемости стала ближе к 0.95, а разброс заметно меньше.

По ЗБЧ и ЦПТ средняя доля покрытий сходится к теоретическому значению 0.95, а её стандартная ошибка убывает как $\sqrt{p(1-p)/N}$ (порядка $1/\sqrt{N}$). Поэтому при увеличении числа симуляций оценка становится стабильнее и ближе к 0.95.


# Задание 2


Построим асимптотический доверительный интервал уровня $1 - \alpha$ для неизвестного сдвига $\mu$ распределения Лапласа с масштабирующим коэффициентом 1.

Отличие от предыдущего задания заключается в том, что интервал с заднным уровнем доверия достигает оценки доверия с ростом размера выборки.

Плотность распределения Лапласа: 
$$ p (x) = \frac{1}{2} e^{- |x - \mu|}$$

Исследуем выборочную медиану $\Mu_n$ нашей выборки с заданным распределением. Она даст более эффективную оценку, в сравнении, например с выборочным средним за счёт меньшей дисперсии, которую найдём дальше.

Дисперсия распределения Лапласа: $2/b^2$. Дисперсия выборочного среднего равна отношению исходной дисперсии к размеру выборки: $2/nb^2$

Вспомним теорему об асимптотике среднего члена вариационного ряда:

$$ \sqrt{n} \frac{\Mu_n - \mu}{\sqrt{1/2 \cdot 1/2}} p(\mu) = \sqrt{n}(\Mu_n - \mu) \rightarrow_d \cal{N} (0, 1), \; \mu - \text{истинная медиана, квантиль } 1/2$$

$p (\mu) = 1/2 $, тогда: 

$$ \Mu_n \sim \cal{N}(\mu, \frac{1}{n})$$

Пусть $\phi_{\alpha/2}$ -- квантиль стантартного нормального распределения уровня $1-\alpha$

В терминах асимптотического доверительного интервала выведем его:

$$ \lim_{n \rightarrow \infty} P (\phi_{\alpha/2} \leq \sqrt{n}(\Mu_n - \mu) \leq \phi_{1 - \alpha/2}) \geq 1 - \alpha$$
$$ \lim_{n \rightarrow \infty} P (\Mu_n + \frac{\phi_{\alpha/2}}{\sqrt{n}} \leq \mu \leq \Mu_n + \frac{\phi_{1-\alpha/2}}{\sqrt{n}}) \geq 1- \alpha$$
Асимптотический доверительный интервал для $\mu$

$$ \mu \in [\Mu_n + \frac{\phi_{\alpha/2}}{\sqrt{n}}; \;\Mu_n + \frac{\phi_{1- \alpha/2}}{\sqrt{n}}] = I$$


In [17]:
import numpy as np
from scipy.stats import norm

mu_true = 2.0
b = 1.0
alpha = 0.05
z = norm.ppf(1 - alpha/2)

rng2 = np.random.default_rng(123)

def coverage_rate_mu(n: int, N: int, chunk_bytes: int = 64_000_000) -> float:
    max_k = max(1, int(chunk_bytes // (8 * n)))
    remain = N
    covered = 0
    while remain > 0:
        k = min(remain, max_k)
        X = rng2.laplace(loc=mu_true, scale=b, size=(k, n))
        med = np.median(X, axis=1)
        half = z / np.sqrt(n)
        lo = med - half
        hi = med + half
        covered += np.count_nonzero((lo <= mu_true) & (mu_true <= hi))
        remain -= k
    return covered / N

n_list = [25, 50, 100, 1000]
N1 = 1000
N2 = 10000

print("Покрытие 95%-асимптотического ДИ для μ:")
for n in n_list:
    p1 = coverage_rate_mu(n, N1)
    p2 = coverage_rate_mu(n, N2)
    se1 = np.sqrt(max(p1*(1-p1), 1e-12) / N1)
    se2 = np.sqrt(max(p2*(1-p2), 1e-12) / N2)
    print(f"n={n:4d} | N={N1:5d}: {p1:.4f}  (±{1.96*se1:.4f} error);  N={N2:5d}: {p2:.4f}  (±{1.96*se2:.4f} error)")


Покрытие 95%-асимптотического ДИ для μ:
n=  25 | N= 1000: 0.9000  (±0.0186 error);  N=10000: 0.9051  (±0.0057 error)
n=  50 | N= 1000: 0.9130  (±0.0175 error);  N=10000: 0.9261  (±0.0051 error)
n= 100 | N= 1000: 0.9250  (±0.0163 error);  N=10000: 0.9281  (±0.0051 error)
n=1000 | N= 1000: 0.9510  (±0.0134 error);  N=10000: 0.9431  (±0.0045 error)
n=1000 | N= 1000: 0.9510  (±0.0134 error);  N=10000: 0.9431  (±0.0045 error)



- На $n = 25$ для 1000 повторов покрытие около 0.90; для 10000 — около 0.905. Оценка стабильна, но заметно ниже 0.95.
- В отличие от точного интервала, здесь используется асимптотический интервал: для конечного $n$ покрытие может отклоняться от номинального уровня.

Причины отклонения:
- Асимптотика медианы даёт $Var(\text{Median}) \approx 1/n$ при $n\to\infty$ и нормальную приближенность, но при $n=25$ аппроксимация может быть грубой.
- С ростом числа симуляций (N) уменьшается Монте‑Карло разброс оценки (закон больших чисел), но смещение самого уровня покрытия (из‑за конечного $n$) сохраняется.

Вывод: для малых/умеренных $n$ асимптотический дов. интервал для $\mu$ в Лапласе может недодерживать уровень 0.95; повышение $n$ или использование более точных методов улучшает покрытие.
